In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

if not tf.__version__.startswith('2'):
    raise ValueError('This code requires TensorFlow V2.x')


In [4]:
(train_data, test_data), info = tfds.load(
    'imdb_reviews',
    split=[tfds.Split.TRAIN, tfds.Split.TEST],
    as_supervised=True,
    with_info=True
)

# Tokenize and pad the reviews
tokenizer = keras.preprocessing.text.Tokenizer(num_words=10000, oov_token='<OOV>')
train_sentences, train_labels = zip(*[(sent.numpy().decode('utf8'), label.numpy()) for sent, label in train_data])
tokenizer.fit_on_texts(train_sentences)
train_sequences = tokenizer.texts_to_sequences(train_sentences)
train_padded = keras.preprocessing.sequence.pad_sequences(train_sequences, maxlen=100, padding='post', truncating='post')

test_sentences, test_labels = zip(*[(sent.numpy().decode('utf8'), label.numpy()) for sent, label in test_data])
test_sequences = tokenizer.texts_to_sequences(test_sentences)
test_padded = keras.preprocessing.sequence.pad_sequences(test_sequences, maxlen=100, padding='post', truncating='post')

train_labels = np.array(train_labels)
test_labels = np.array(test_labels)


In [5]:
model = keras.models.Sequential([
    keras.layers.Embedding(10000, 64),
    keras.layers.SimpleRNN(64),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

history = model.fit(train_padded, train_labels, epochs=10, validation_data=(test_padded, test_labels))



In [6]:
history = model.fit(train_padded, train_labels, epochs=10, validation_data=(test_padded, test_labels))


Epoch 1/10
782/782 [==============================] - 18s 22ms/step - loss: 0.6941 - accuracy: 0.5172 - val_loss: 0.6925 - val_accuracy: 0.5174
Epoch 2/10
782/782 [==============================] - 16s 21ms/step - loss: 0.6650 - accuracy: 0.5926 - val_loss: 0.7660 - val_accuracy: 0.5042
Epoch 3/10
782/782 [==============================] - 16s 21ms/step - loss: 0.6054 - accuracy: 0.6712 - val_loss: 0.7146 - val_accuracy: 0.5406
Epoch 4/10
782/782 [==============================] - 16s 21ms/step - loss: 0.5295 - accuracy: 0.7248 - val_loss: 0.7645 - val_accuracy: 0.5749
Epoch 5/10
782/782 [==============================] - 19s 24ms/step - loss: 0.4356 - accuracy: 0.7876 - val_loss: 0.7763 - val_accuracy: 0.6078
Epoch 6/10
782/782 [==============================] - 16s 21ms/step - loss: 0.3917 - accuracy: 0.8140 - val_loss: 0.7221 - val_accuracy: 0.5491
Epoch 7/10
782/782 [==============================] - 18s 23ms/step - loss: 0.4030 - accuracy: 0.8115 - val_loss: 0.9223 - val_accuracy:

In [7]:
model = keras.models.Sequential([
    keras.layers.Embedding(10000, 32), # Reduced embedding dimension
    keras.layers.Dropout(0.5), # Added dropout
    keras.layers.SimpleRNN(32, kernel_regularizer=keras.regularizers.l2(0.001)), # L2 regularization
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Adding early stopping callback
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

history = model.fit(train_padded, train_labels, epochs=10, validation_data=(test_padded, test_labels), callbacks=[early_stop])


Epoch 1/10
782/782 [==============================] - 12s 14ms/step - loss: 0.7146 - accuracy: 0.5183 - val_loss: 0.7044 - val_accuracy: 0.5380
Epoch 2/10
782/782 [==============================] - 11s 14ms/step - loss: 0.6914 - accuracy: 0.5749 - val_loss: 0.6794 - val_accuracy: 0.6038
Epoch 3/10
782/782 [==============================] - 11s 15ms/step - loss: 0.6105 - accuracy: 0.6842 - val_loss: 0.5641 - val_accuracy: 0.7507
Epoch 4/10
782/782 [==============================] - 11s 15ms/step - loss: 0.4341 - accuracy: 0.8148 - val_loss: 0.5487 - val_accuracy: 0.7716
Epoch 5/10
782/782 [==============================] - 11s 14ms/step - loss: 0.3399 - accuracy: 0.8662 - val_loss: 0.5917 - val_accuracy: 0.7576
Epoch 6/10
782/782 [==============================] - 12s 15ms/step - loss: 0.3198 - accuracy: 0.8720 - val_loss: 0.6870 - val_accuracy: 0.6710
Epoch 7/10
782/782 [==============================] - 12s 15ms/step - loss: 0.4039 - accuracy: 0.8154 - val_loss: 0.7807 - val_accuracy:

In [8]:
from keras.layers import Bidirectional, LSTM

model = keras.models.Sequential([
    keras.layers.Embedding(10000, 32),
    Bidirectional(LSTM(32, dropout=0.5, recurrent_dropout=0.5, kernel_regularizer=keras.regularizers.l2(0.001))),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss="binary_crossentropy", optimizer=keras.optimizers.Adam(learning_rate=0.001), metrics=["accuracy"])

early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

history = model.fit(train_padded, train_labels, epochs=10, validation_data=(test_padded, test_labels), callbacks=[early_stop])


Epoch 1/10
782/782 [==============================] - 98s 120ms/step - loss: 0.5122 - accuracy: 0.7501 - val_loss: 0.4354 - val_accuracy: 0.8051
Epoch 2/10
782/782 [==============================] - 96s 123ms/step - loss: 0.3447 - accuracy: 0.8626 - val_loss: 0.4804 - val_accuracy: 0.8075
Epoch 3/10
782/782 [==============================] - 98s 125ms/step - loss: 0.2945 - accuracy: 0.8852 - val_loss: 0.4370 - val_accuracy: 0.8124
Epoch 4/10
782/782 [==============================] - 97s 123ms/step - loss: 0.2665 - accuracy: 0.8990 - val_loss: 0.4887 - val_accuracy: 0.8044


In [9]:
from keras.layers import Conv1D, MaxPooling1D

model = keras.models.Sequential([
    keras.layers.Embedding(10000, 32),
    Conv1D(32, 7, activation='relu'),
    MaxPooling1D(5),
    Bidirectional(LSTM(32, return_sequences=True, dropout=0.5, recurrent_dropout=0.5, kernel_regularizer=keras.regularizers.l2(0.001))),
    Bidirectional(LSTM(32, dropout=0.5, recurrent_dropout=0.5, kernel_regularizer=keras.regularizers.l2(0.001))),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss="binary_crossentropy", optimizer=keras.optimizers.RMSprop(learning_rate=0.001), metrics=["accuracy"])

early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
checkpoint = keras.callbacks.ModelCheckpoint("best_model.h5", save_best_only=True)

history = model.fit(train_padded, train_labels, epochs=10, validation_data=(test_padded, test_labels), callbacks=[early_stop, checkpoint])


Epoch 1/10
782/782 [==============================] - 47s 52ms/step - loss: 0.5050 - accuracy: 0.7582 - val_loss: 0.4335 - val_accuracy: 0.8117
Epoch 2/10
  3/782 [..............................] - ETA: 36s - loss: 0.4024 - accuracy: 0.8333

/Users/muhammadluay/opt/miniconda3/envs/tensyflow/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


782/782 [==============================] - 41s 52ms/step - loss: 0.3505 - accuracy: 0.8554 - val_loss: 0.4137 - val_accuracy: 0.8125
Epoch 3/10
782/782 [==============================] - 41s 52ms/step - loss: 0.3053 - accuracy: 0.8781 - val_loss: 0.3992 - val_accuracy: 0.8236
Epoch 4/10
782/782 [==============================] - 42s 53ms/step - loss: 0.2723 - accuracy: 0.8951 - val_loss: 0.4099 - val_accuracy: 0.8208
Epoch 5/10
782/782 [==============================] - 42s 53ms/step - loss: 0.2429 - accuracy: 0.9115 - val_loss: 0.5047 - val_accuracy: 0.7852
Epoch 6/10
782/782 [==============================] - 41s 53ms/step - loss: 0.2078 - accuracy: 0.9269 - val_loss: 0.4410 - val_accuracy: 0.8169
